<a href="https://colab.research.google.com/github/tuenv193176/FOM/blob/main/testFOM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
import re
import numpy as np



In [3]:
# Load the data
file_path = '/content/data.csv'
data = pd.read_csv(file_path)

# Define the frequency ranges
freq_ranges = [
    (2.35, 2.47),
    (5.15, 5.27),
    (5.76, 5.88)
]

# Extract the frequencies and their corresponding columns
frequencies = data['Freq [GHz]']
columns = data.columns[1:]

# Initialize an empty DataFrame to store FOM values
fom_data = pd.DataFrame()

# Function to calculate FOM for a given frequency range
def calculate_fom(freq_range):
    start_freq, end_freq = freq_range
    mask = (frequencies >= start_freq) & (frequencies <= end_freq)
    return data.loc[mask, columns].abs().sum()

# Calculate FOM for each frequency range and store in fom_data
for freq_range in freq_ranges:
    fom_data[f'FOM_{freq_range[0]}_{freq_range[1]}'] = calculate_fom(freq_range)

# Sum the FOM values for all frequency ranges to get the total FOM
fom_data['Total_FOM'] = fom_data.sum(axis=1)

# Function to extract parameters from a column name
def extract_parameters(column_name):
    match = re.search(r"ls1='([\d.]+)mm' ls2='([\d.]+)mm' ls3='([\d.]+)mm' ws1='([\d.]+)mm' ws2='([\d.]+)mm' ws3='([\d.]+)mm'", column_name)
    if match:
        return [float(match.group(i)) for i in range(1, 7)]
    return [None] * 6

# Extract parameters for each column
params_list = [extract_parameters(col) for col in columns]

# Create a DataFrame for the parameters
param_data = pd.DataFrame(params_list, columns=['ls1', 'ls2', 'ls3', 'ws1', 'ws2', 'ws3'])

# Ensure the indices match between param_data and fom_data
param_data.index = fom_data.index

# Check for invalid extractions and filter them out
valid_rows = param_data.notnull().all(axis=1)
param_data = param_data[valid_rows].reset_index(drop=True)
fom_data = fom_data[valid_rows].reset_index(drop=True)

# Combine parameters with FOM values
final_data = pd.concat([param_data, fom_data['Total_FOM']], axis=1)

# Display the first few rows of the final_data to verify
print(final_data.head())

# Prepare the input and output data
X = final_data[['ls1', 'ls2', 'ls3', 'ws1', 'ws2', 'ws3']]
y = final_data['Total_FOM']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()

    ls1  ls2  ls3   ws1  ws2   ws3   Total_FOM
0  7.25  8.5  7.5  0.50  1.5  0.25   82.127649
1  7.50  8.5  7.5  0.50  1.5  0.25   83.729922
2  7.75  8.5  7.5  0.50  1.5  0.25  101.203508
3  8.00  8.5  7.5  0.50  1.5  0.25  130.309819
4  7.25  8.5  7.5  0.75  1.5  0.25   93.533662


In [5]:
X

,ls1,ls2,ls3,ws1,ws2,ws3
0,7.25,8.5,7.50,0.50,1.5,0.25
1,7.50,8.5,7.50,0.50,1.5,0.25
2,7.75,8.5,7.50,0.50,1.5,0.25
3,8.00,8.5,7.50,0.50,1.5,0.25
4,7.25,8.5,7.50,0.75,1.5,0.25
...,...,...,...,...,...,...
1939,7.75,9.5,8.25,0.75,2.0,0.75
1940,8.00,9.5,8.25,0.75,2.0,0.75
1941,7.50,9.5,8.25,1.00,2.0,0.75
1942,7.75,9.5,8.25,1.00,2.0,0.75


In [6]:
y

0        82.127649
1        83.729922
2       101.203508
3       130.309819
4        93.533662
           ...    
1939    225.383722
1940    169.564996
1941    250.624889
1942    209.685021
1943     94.111705
Name: Total_FOM, Length: 1944, dtype: float64

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.regularizers import L1

# Build the ANN model with 5 hidden layers
model = Sequential()
model.add(Dense(64, input_dim=6, activation='relu', kernel_regularizer=L1(0.001)))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))  # Regression output

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

# Train the model
history = model.fit(X_train_scaled, y_train_scaled, epochs=500, batch_size=32, validation_data=(X_test_scaled, y_test_scaled))


Epoch 1/500
49/49 [==============================] - 2s 9ms/step - loss: 0.7383 - mse: 0.6840 - mae: 0.6640 - val_loss: 0.4728 - val_mse: 0.4191 - val_mae: 0.5224
Epoch 2/500
49/49 [==============================] - 0s 4ms/step - loss: 0.4851 - mse: 0.4319 - mae: 0.5163 - val_loss: 0.3866 - val_mse: 0.3339 - val_mae: 0.4663
Epoch 3/500
49/49 [==============================] - 0s 5ms/step - loss: 0.4547 - mse: 0.4024 - mae: 0.4970 - val_loss: 0.3521 - val_mse: 0.3004 - val_mae: 0.4394
Epoch 4/500
49/49 [==============================] - 0s 5ms/step - loss: 0.4297 - mse: 0.3785 - mae: 0.4822 - val_loss: 0.3261 - val_mse: 0.2754 - val_mae: 0.4215
Epoch 5/500
49/49 [==============================] - 0s 6ms/step - loss: 0.3705 - mse: 0.3203 - mae: 0.4383 - val_loss: 0.2897 - val_mse: 0.2399 - val_mae: 0.3875
Epoch 6/500
49/49 [==============================] - 0s 8ms/step - loss: 0.3631 - mse: 0.3137 - mae: 0.4294 - val_loss: 0.3324 - val_mse: 0.2834 - val_mae: 0.4246
Epoch 7/500
49/49 [===

In [8]:
# Evaluate the model
test_loss, test_mse, test_mae = model.evaluate(X_test_scaled, y_test_scaled)
print(f"Test Loss: {test_loss:.4f}, Test MSE: {test_mse:.4f}, Test MAE: {test_mae:.4f}")


13/13 [==============================] - 0s 2ms/step - loss: 0.3166 - mse: 0.2935 - mae: 0.4025
Test Loss: 0.3166, Test MSE: 0.2935, Test MAE: 0.4025


In [11]:

y_pred_scaled = model.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled).flatten()

num_samples = 10
indices = np.random.choice(len(y_test), num_samples, replace=False)

comparison_df = pd.DataFrame({
    'Actual FOM': y_test.values[indices],
    'Predicted FOM': y_pred[indices]
})

print(comparison_df)

13/13 [==============================] - 0s 2ms/step
   Actual FOM  Predicted FOM
0  319.097836     351.723236
1  332.645963     208.363266
2  281.196626     283.778259
3  487.077641     444.538757
4  309.843718     282.792664
5  144.393275     202.354324
6  287.172675     289.529114
7  371.605775     438.337067
8  296.889970     339.837097
9  282.990946     277.904602
